In [ ]:
#importação das bibliotecas
from dotenv import load_dotenv
import os
load_dotenv()

True

In [ ]:
import os

from groq import Groq

#Requisição da API
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)
#iniciando chat com o prompt: Explain the importance of fast language models.
print(chat_completion.choices[0].message.content)

Fast language models are revolutionizing the field of natural language processing (NLP) and have numerous applications in various industries. The importance of fast language models can be summarized as follows:

1. **Improved Efficiency**: Fast language models can process and analyze large amounts of text data quickly, enabling applications such as text classification, language translation, and sentiment analysis to be performed in real-time. This efficiency is crucial for applications that require instantaneous responses, such as chatbots, virtual assistants, and customer service platforms.
2. **Enhanced User Experience**: Fast language models enable developers to build more responsive and interactive applications, providing a better user experience. For instance, a fast language model can quickly generate human-like responses to user queries, facilitating more natural and engaging conversations.
3. **Increased Scalability**: Fast language models can handle large volumes of text data 

In [ ]:
#Criando a classe Agent com as entradas client e system(prompt do sistema)
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []  #Esse campo será o histórico do chat. Nele as mensagens enviadas serão armazenadas, assim como os pensamentos do modelo.
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})
    #Essa função permite que a instância da classe seja tradada como uma função. A intância poderá ser chamada com o argumento message, que é o prompt do usuário
    def __call__(self, message=""):
        if message:   #se houver message adicionar ele ao histórico
            self.messages.append({"role":"user", "content": message})
        result = self.execute()  #executar a função execute, que irá iniciar o chat
        self.messages.append({"role": "assistant", "content": result}) # adicionar o resultado às mensagens
        return result
    
    def execute(self): 
        completion = client.chat.completions.create(    #criando o chat com o modelo llama3 e iniciando ele com as mensagens presentes no self.messages
            messages = self.messages,
            model = "llama3-70b-8192"
        )
        return completion.choices[0].message.content

In [ ]:
#prompt do sistema
#definindo o fluxo de funcionamento do agente: pensamento, ação, pausa, observação e resposta(caso haja)
#nesse prompt nós apresentamos ao agente as ferramentas que lhe estão disponíveis.
system_prompt = """

You run in a loop of Thought, Action, Pause, Observation.               
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you habe been asked.
Use Action to run one of the action available to you - then return PAUSE.
Observation will be the result of running those actions.

You available actions are: 

calculat:
e.g. calculate: 4 * 7 / 3
Runs a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary

get_planet_mass:        
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I neet to fin te mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this: 

Observation: 5.972e24

Thought: I need to multiply this by 2 
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 5.972e24

Thoutht: I need to multiply this by 2 
Action: calculate  5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1.1944x10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,944x10e25. 

Now its your turn:

""".strip()

In [ ]:
# Ferramentas (Tools) 
def calculate(operation): #calculadora
    return eval(operation)

def get_planet_mass(planet) -> float: #encontrar a massa do planeta. Se o argumento corresponder a alguma string a função retornará o valor da string em massa
    match planet.lower():
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case _:
            return 0.0

In [ ]:
neil_tyson = Agent(client, system_prompt)       #Instância da classe

In [ ]:
result = neil_tyson("what is the mass of Earth times 5?") #Primeiro prompt
print(result)

Thought: I need to find the mass of Earth.


In [ ]:
neil_tyson.messages #histórico com o primeiro prompt

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I neet to fin te mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2 \nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThoutht: I need to mul

In [ ]:
result = neil_tyson()  #continuando o loop. Como o pensamento foi feito na primeira execução, agora chegamos no passo da ação
print(result)

Action: calculate: 5.972e24 * 5 
PAUSE


In [ ]:
observation = get_planet_mass("Earth")   #executando a função para que o agente receba o valor dela como observação
print(observation)

5.972e+24


In [ ]:
next_prompt = f"Observation: {observation}"   #enviando a observação 
result = neil_tyson(next_prompt) # código seguirá a execução das ultimmas células até que a resposta seja obtida.
print(result)

Thought: I need to multiply this by 5.


In [13]:
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I neet to fin te mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2 \nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThoutht: I need to mul

In [ ]:
#importando a biblioteca re para a identificação da ação e do argumento
import re
#agora a ideia é produzir um agente que funcione em loop, descartando a necessidade de ficar executando células manualmente
def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    while i < max_iterations: #define que o loop será executado até que i ( o contador ) seja menor que o número máximo de iterações
        i += 1 #incrimenta 1 a i após cada iteração
        result = agent(next_prompt) #Após cada loop result assume um valor diferente. No primeiro loop result está definido como query
        print(result)
        #Se pause e action estiver no resultado a biblioteca re irá achar qual é a ação e qual é o argumento da ação. 
        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0] # ferramenta encontrada
            arg = action[0][1] #argumento do ferramenta

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')") #aqui a "observação" será salva e adicionada ao próximo prompt
                next_prompt = f"Observation: {result_tool}"

            else:   #caso não haja ferramenta haverá um erro
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue #continua para a próxima iteração

        if "Answer" in result: #se o termo answer estiver nos resultados o loop é encerrado e a resposta estará armazenada em result
            break



In [15]:
agent_loop(10, system = system_prompt, query="what is the mass of the Earth plus the mass of Mercury, and All of it times 5")

Thought: I need to find the mass of the Earth and the mass of Mercury, and then add them together before multiplying the result by 5.
Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: I have the mass of the Earth, now I need to get the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I have the masses of both the Earth and Mercury, now I need to add them together.
Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: I have the sum of the masses, now I need to multiply it by 5.
Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have the final result, which is the mass of the Earth plus the mass of Mercury, multiplied by 5.

Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.15025x10e25.
